In [571]:
import pandas as pd
pd.set_option('display.max_columns', None)

## Load Dataset

In [572]:
players_df = pd.read_csv('../../data/processed/players_information.csv')
players_df = players_df.sort_values(by='season').reset_index(drop=True)
players_df

,name,position,nationality,team,season
0,Nathan Aké,Defender,Netherlands,Bournemouth,2016/17
1,Conor Masterson,Defender,Ireland,Liverpool,2016/17
2,Daniel Cleary,Defender,Ireland,Liverpool,2016/17
3,Joseph Maguire,Defender,England,Liverpool,2016/17
4,Connor Randall,Defender,England,Liverpool,2016/17
...,...,...,...,...,...
8972,Marcus Tavernier,Midfielder,England,Bournemouth,2024/25
8973,Alex Scott,Midfielder,England,Bournemouth,2024/25
8974,Ryan Christie,Midfielder,Scotland,Bournemouth,2024/25
8975,Sasa Lukic,Midfielder,Serbia,Fulham,2024/25


In [573]:
import re

def clean_name(name):
    cleaned_name = re.sub(r'_\d+$', '', name)
    cleaned_name = cleaned_name.replace('_', ' ')
    return cleaned_name

In [574]:
def fixing_pos_team(df, name, position, team):
    df.loc[df['name'] == name, ['position', 'team']] = [position, team]

In [575]:
from rapidfuzz import process, fuzz
import pandas as pd

# Fungsi untuk membalik nama pemain Korea Selatan
def reverse_korean_name(name):
    parts = name.split(' ')
    if len(parts) == 2:
        return f"{parts[1]} {parts[0]}"
    return name

# Terapkan fungsi pada pemain dengan kewarganegaraan 'South Korea'
def apply_reverse_to_korean_players(player_info):
    korean_players = player_info['nationality'] == 'South Korea'
    player_info.loc[korean_players, 'name'] = player_info.loc[korean_players, 'name'].apply(reverse_korean_name)
    return player_info

# Modifikasi fungsi untuk mencocokkan nama pemain dan menambahkan kolom 'position' dan 'team'
def match_and_add_columns(row, players_df):
    # Menerapkan perubahan format nama pemain dari Korea Selatan
    players_df = apply_reverse_to_korean_players(players_df)
    
    # Menggunakan rapidfuzz untuk menemukan nama terdekat berdasarkan kolom 'name'
    match = process.extractOne(row['name'], players_df['name'], scorer=fuzz.ratio)
    
    # Jika skor kesamaan (threshold) lebih besar atau sama dengan 30, lanjutkan ke validasi lanjutan
    if match and match[1] >= 60:  
        matched_row = players_df[players_df['name'] == match[0]].iloc[0]
        
        # Mengganti nilai kolom 'name' pada df_1617 dengan 'name' dari players_df
        row['name'] = matched_row['name']
        
        # Mengembalikan posisi dan tim dari players_df
        return pd.Series([matched_row['position'], matched_row['team']])
    
    # Jika tidak ada kecocokan, kembalikan None untuk posisi dan tim
    return pd.Series([None, None])

In [576]:
team_map = {
    'AFC Bournemouth':'Bournemouth', 
    'Arsenal FC':'Arsenal', 
    'Brighton':'Brighton & Hove Albion', 
    'Burnley FC':'Burnley', 
    'Chelsea FC':'Chelsea',
    'Everton FC':'Everton', 
    'Fulham FC':'Fulham',
    'Leeds':'Leeds United', 
    'Leicester':'Leicester City',
    'Liverpool FC':'Liverpool', 
    'Luton':'Luton Town', 
    'Man City':'Manchester City',
    'Man Utd':'Manchester United', 
    'Middlesbrough FC':'Middlesbrough', 
    'Newcastle':'Newcastle United', 
    'Norwich':'Norwich City', 
    "Nott'm Forest":'Nottingham Forest', 
    'Sheffield Utd':'Sheffield United',  
    'Southampton FC':'Southampton', 
    'Spurs':'Tottenham Hotspur',
    'Sunderland AFC':'Sunderland', 
    'Watford FC':'Watford', 
    'West Brom':'West Bromwich Albion',
    'West Ham':'West Ham United',
    'Wolves':'Wolverhampton Wanderers',
    'Huddersfield':'Huddersfield Town',
    'Hull':'Hull City',
    'Stoke':'Stoke City',
    'Swansea':'Swansea City',
    'Cardiff':'Cardiff City',
}

def replace_team_value(df):
    df['team'] = df['team'].replace(team_map)

### 2016-17

In [577]:
url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/merged_gw.csv"
df_1617 = pd.read_csv(url, encoding='ISO-8859-1')

In [578]:
df_1617.head(3)

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded,goals_scored,ict_index,id,influence,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,offside,open_play_crosses,opponent_team,own_goals,penalties_conceded,penalties_missed,penalties_saved,recoveries,red_cards,round,saves,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,GW
0,Aaron_Cresswell,0,0,0,0,0,0,0,0,0,0.0,0,0,454,0,0,10,0,0,0,0.0,454,0.0,0,2016-08-15T19:00:00Z,15 Aug 20:00,0,0,0,0,0,4,0,0,0,0,0,0,1,0,14023,0,0,0,1,2,0,0,0,0,0,55,False,0,0,1
1,Aaron_Lennon,0,3,0,0,0,6,0,1,2,0.3,0,0,142,0,0,3,0,0,0,0.9,142,8.2,0,2016-08-13T14:00:00Z,13 Aug 15:00,0,0,15,0,0,17,0,0,0,0,1,0,1,0,13918,1,2,0,1,1,0,1,0,0,0,60,True,0,0,1
2,Aaron_Ramsey,0,26,0,0,0,5,0,2,22,4.9,1,0,16,0,0,8,0,3,0,3.0,16,2.2,0,2016-08-14T15:00:00Z,14 Aug 16:00,0,0,60,0,0,9,0,0,0,0,2,0,1,0,163170,2,0,1,4,3,23,2,0,0,0,80,True,0,0,1


In [579]:
players_1617_df = players_df[players_df['season'] == '2016/17']

# Terapkan fungsi untuk setiap baris di df_1617
df_1617['name'] = df_1617['name'].apply(clean_name)

# Pastikan df_1617 memiliki kolom 'position' dan 'team' terlebih dahulu
df_1617[['position', 'team']] = df_1617.apply(match_and_add_columns, players_df=players_1617_df, axis=1)

In [580]:
df_1617[df_1617['position'].isnull()]['name'].sort_values().unique()

array(['Adrián San Miguel del Castillo', 'David Luiz Moreira Marinho',
       'Eduardo Dos Reis Carvalho', 'Fabio Pereira da Silva',
       'Fernando Francisco Reges', 'Gabriel Armando de Abreu',
       'Jose Angel Esmoris Tasende', 'Jose Luis Mato Sanmartín',
       'Leiva Lucas', 'Manuel Agudo Durán', 'Marco Van Ginkel',
       'Michael Simões Domingues', 'Olayinka Fredrick Oladotun Ladapo',
       'Oscar dos Santos Emboaba Junior',
       'Robert Kenedy Nunes do Nascimento', 'Takuma Asano',
       'Willian Borges Da Silva'], dtype=object)

In [581]:
fixing_pos_team(df_1617, 'Adrián San Miguel del Castillo', 'Goalkeeper', 'West Ham United')
fixing_pos_team(df_1617, 'David Luiz Moreira Marinho', 'Defender', 'Chelsea')
fixing_pos_team(df_1617, 'Eduardo Dos Reis Carvalho', 'Goalkeeper', 'Chelsea')
fixing_pos_team(df_1617, 'Fabio Pereira da Silva', 'Defender', 'Manchester United')
fixing_pos_team(df_1617, 'Fernando Francisco Reges', 'Midfielder', 'Manchester City')
fixing_pos_team(df_1617, 'Gabriel Armando de Abreu', 'Defender', 'Arsenal')
fixing_pos_team(df_1617, 'Jose Angel Esmoris Tasende', 'Defender', 'Manchester City')
fixing_pos_team(df_1617, 'Leiva Lucas', 'Midfielder', 'Liverpool')
fixing_pos_team(df_1617, 'Manuel Agudo Durán', 'Forward', 'Manchester City')
fixing_pos_team(df_1617, 'Jose Luis Mato Sanmartín', 'Forward', 'Stoke City')
fixing_pos_team(df_1617, 'Marco Van Ginkel', 'Midfielder', 'Chelsea')
fixing_pos_team(df_1617, 'Michael Simões Domingues', 'Goalkeeper', 'Sunderland')
fixing_pos_team(df_1617, 'Olayinka Fredrick Oladotun Ladapo', 'Forward', 'Crystal Palace')
fixing_pos_team(df_1617, 'Oscar dos Santos Emboaba Junior', 'Midfielder', 'Chelsea')
fixing_pos_team(df_1617, 'Robert Kenedy Nunes do Nascimento', 'Midfielder', 'Chelsea')
fixing_pos_team(df_1617, 'Takuma Asano', 'Forward', 'Arsenal')
fixing_pos_team(df_1617, 'Willian Borges Da Silva', 'Forward', 'Chelsea')

### 2017-18

In [582]:
url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/merged_gw.csv"
df_1718 = pd.read_csv(url, encoding='ISO-8859-1')

In [583]:
df_1718.head(3)

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded,goals_scored,ict_index,id,influence,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,offside,open_play_crosses,opponent_team,own_goals,penalties_conceded,penalties_missed,penalties_saved,recoveries,red_cards,round,saves,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,GW
0,Aaron_Cresswell,0,10,0,0,0,3,0,0,9,0.6,0,0,443,0,0,9,0,2,0,1.9,443,0.4,0,2017-08-13T15:00:00Z,13 Aug 16:00,0,0,9,0,0,12,0,0,0,0,0,0,1,0,25136,0,0,0,0,4,18,0,0,0,0,50,False,0,0,1
1,Aaron_Lennon,0,0,0,0,0,0,0,0,0,0.0,0,0,153,0,0,5,0,0,0,0.0,153,0.0,0,2017-08-12T14:00:00Z,12 Aug 15:00,0,0,0,0,0,15,0,0,0,0,0,0,1,0,4681,0,0,0,0,1,0,0,0,0,0,55,True,0,0,1
2,Aaron_Mooy,1,52,0,0,0,22,1,5,41,46.9,0,0,172,0,0,4,0,0,0,8.7,172,40.2,3,2017-08-12T14:00:00Z,12 Aug 15:00,0,0,90,0,1,6,0,0,0,0,5,0,1,0,59955,2,0,0,3,0,0,6,0,0,0,55,False,0,0,1


In [584]:
players_1718_df = players_df[players_df['season'] == '2017/18']

df_1718['name'] = df_1718['name'].apply(clean_name)
df_1718[['position', 'team']] = df_1718.apply(match_and_add_columns, players_df=players_1718_df, axis=1)

In [585]:
df_1718[df_1718['team'].isnull()]['name'].sort_values().unique()

array(['Adrien Sebastian Perruchet Silva',
       'Adrián San Miguel del Castillo',
       'Bernardo Mota Veiga de Carvalho e Silva', 'Bojan Krkic Perez',
       'Bruno Saltor Grau', 'David Luiz Moreira Marinho',
       'Eduardo dos Reis Carvalho', 'Emerson Palmieri dos Santos',
       'Fernando Francisco Reges', 'Gabriel Armando de Abreu',
       'Javier Hernández Balcázar', 'Jose Luis Mato Sanmartín',
       'João Mário Naval Costa Eduardo', 'Lucas Rodrigues Moura da Silva',
       'Manuel Agudo Durán', 'Olayinka Fredrick Oladotun Ladapo',
       'Pedro Rodríguez Ledesma', 'Robert Kenedy Nunes do Nascimento',
       'Willian Borges Da Silva'], dtype=object)

In [586]:
fixing_pos_team(df_1718, 'Adrien Sebastian Perruchet Silva', 'Midfielder', 'Leicester City')
fixing_pos_team(df_1718, 'Adrián San Miguel del Castillo', 'Goalkeeper', 'West Ham United')
fixing_pos_team(df_1718, 'Bernardo Mota Veiga de Carvalho e Silva', 'Midfielder', 'Manchester City')
fixing_pos_team(df_1718, 'Bojan Krkic Perez', 'Forward', 'Stoke City')
fixing_pos_team(df_1718, 'Bruno Saltor Grau', 'Defender', 'Brighton & Hove Albion')
fixing_pos_team(df_1718, 'David Luiz Moreira Marinho', 'Defender', 'Chelsea')

fixing_pos_team(df_1718, 'Ederson Santana de Moraes', 'Goalkeeper', 'Manchester City')
fixing_pos_team(df_1718, 'Eduardo dos Reis Carvalho', 'Goalkeeper', 'Chelsea')
fixing_pos_team(df_1718, 'Emerson Palmieri dos Santos', 'Defender', 'Chelsea')
fixing_pos_team(df_1718, 'Fernando Francisco Reges', 'Midfielder', 'Manchester City')
fixing_pos_team(df_1718, 'Gabriel Armando de Abreu', 'Defender', 'Arsenal')

fixing_pos_team(df_1718, 'Javier Hernández Balcázar', 'Forward', 'West Ham United')
fixing_pos_team(df_1718, 'Jose Luis Mato Sanmartín', 'Forward', 'Stoke City')
fixing_pos_team(df_1718, 'João Mário Naval Costa Eduardo', 'Midfielder', 'West Ham United')
fixing_pos_team(df_1718, 'Manuel Agudo Durán', 'Forward', 'Manchester City')

fixing_pos_team(df_1718, 'Lucas Rodrigues Moura da Silva', 'Forward', 'Tottenham Hotspur')
fixing_pos_team(df_1718, 'Olayinka Fredrick Oladotun Ladapo', 'Forward', 'Crystal Palace')

fixing_pos_team(df_1718, 'Pedro Rodríguez Ledesma', 'Forward', 'Chelsea')
fixing_pos_team(df_1718, 'Robert Kenedy Nunes do Nascimento', 'Midfielder', 'Chelsea')
fixing_pos_team(df_1718, 'Willian Borges Da Silva', 'Forward', 'Chelsea')

In [587]:
df_1718[df_1718['team'].isnull()]['name'].sort_values().unique()

array([], dtype=object)

### 2018-19

In [588]:
url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/merged_gw.csv"
df_1819 = pd.read_csv(url, encoding='ISO-8859-1')

In [589]:
df_1819.head(3)

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded,goals_scored,ict_index,id,influence,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,offside,open_play_crosses,opponent_team,own_goals,penalties_conceded,penalties_missed,penalties_saved,recoveries,red_cards,round,saves,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,GW
0,Aaron_Cresswell_402,0,0,0,0,0,0,0,0,0,0.0,0,0,402,0,0,5,0,0,0,0.0,402,0.0,0,2018-08-12T12:30:00Z,12 Aug 13:30,0,0,0,0,0,12,0,0,0,0,0,0,1,0,103396,0,0,0,0,4,0.0,0,0,0,0,55,False,0,0,1
1,Aaron_Lennon_83,0,22,0,1,0,6,1,1,17,12.3,0,0,83,0,0,8,1,0,0,3.9,83,10.0,0,2018-08-12T12:30:00Z,12 Aug 13:30,0,0,90,0,1,16,0,0,0,0,2,0,1,0,15138,1,2,0,0,0,17.0,3,0,0,0,50,False,0,0,1
2,Aaron_Mooy_199,0,51,0,0,0,24,0,2,40,18.2,1,0,199,0,0,4,1,3,0,3.8,199,20.2,1,2018-08-11T14:00:00Z,11 Aug 15:00,0,0,90,0,0,6,0,0,0,0,11,0,1,0,192110,1,6,0,3,0,0.0,2,0,0,0,55,True,0,0,1


In [590]:
players_1819_df = players_df[players_df['season'] == '2018/19']

# Terapkan fungsi untuk setiap baris di df_1819
df_1819['name'] = df_1819['name'].apply(clean_name)
df_1819[['position', 'team']] = df_1819.apply(match_and_add_columns, players_df=players_1819_df, axis=1)

In [591]:
df_1819['team'].sort_values().unique()

array(['Arsenal', 'Bournemouth', 'Brighton & Hove Albion', 'Burnley',
       'Cardiff City', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham',
       'Huddersfield Town', 'Leicester City', 'Liverpool',
       'Manchester City', 'Manchester United', 'Newcastle United',
       'Southampton', 'Tottenham Hotspur', 'Watford', 'West Ham United',
       'Wolverhampton Wanderers', None], dtype=object)

In [592]:
df_1819[df_1819['team'].isnull()]['name'].sort_values().unique()

array(['Abd-Al-Ali Morakinyo Olaposi Koiki',
       'Addji Keaninkin Marc-Israel Guehi',
       'Adrien Sebastian Perruchet Silva',
       'Adrián San Miguel del Castillo',
       'Alexandre Nascimento Costa Silva', 'André Filipe Tavares Gomes',
       'Bernard Anício Caldeira Duarte',
       'Bernardo Fernandes da Silva Junior',
       'Bernardo Mota Veiga de Carvalho e Silva',
       'Bonatini Lohner Maia Bonatini', 'Bruno Saltor Grau',
       'David Luiz Moreira Marinho', 'Emerson Palmieri dos Santos',
       'Fabio Henrique Tavares', 'Fabricio Agosto Ramírez',
       'Frederico Rodrigues de Paula Santos', 'Javier Hernández Balcázar',
       'Jorge Luiz Frello Filho', 'Jose Luis Mato Sanmartín',
       'João Filipe Iria Santos Moutinho',
       'Lucas Rodrigues Moura da Silva', 'Pedro Rodríguez Ledesma',
       'Robert Kenedy Nunes do Nascimento', 'Rui Pedro da Rocha Fonte',
       'Rui Pedro dos Santos Patrício', 'Rúben Diogo da Silva Neves',
       'Rúben Gonçalo Silva Nascimento 

In [593]:
fixing_pos_team(df_1819, 'Abd-Al-Ali Morakinyo Olaposi Koiki', 'Midfielder', 'Burnley')
fixing_pos_team(df_1819, 'Addji Keaninkin Marc-Israel Guehi', 'Defender', 'Crystal Palace')
fixing_pos_team(df_1819, 'Adrien Sebastian Perruchet Silva', 'Midfielder', 'Leicester City')
fixing_pos_team(df_1819, 'Adrián San Miguel del Castillo', 'Goalkeeper', 'West Ham United')
fixing_pos_team(df_1819, 'Alexandre Nascimento Costa Silva', 'Forward', 'West Ham United')
fixing_pos_team(df_1819, 'André Filipe Tavares Gomes', 'Midfielder', 'Everton')
fixing_pos_team(df_1819, 'Bernard Anício Caldeira Duarte', 'Forward', 'Everton')
fixing_pos_team(df_1819, 'Bernardo Fernandes da Silva Junior', 'Defender', 'Brighton & Hove Albion')
fixing_pos_team(df_1819, 'Bernardo Mota Veiga de Carvalho e Silva', 'Midfielder', 'Manchester City')
fixing_pos_team(df_1819, 'Bonatini Lohner Maia Bonatini', 'Forward', 'Wolverhampton Wanderers')
fixing_pos_team(df_1819, 'Bruno Saltor Grau', 'Defender', 'Brighton & Hove Albion')
fixing_pos_team(df_1819, 'David Luiz Moreira Marinho', 'Defender', 'Chelsea')
fixing_pos_team(df_1819, 'Ederson Santana de Moraes', 'Goalkeeper', 'Manchester City')
fixing_pos_team(df_1819, 'Emerson Palmieri dos Santos', 'Defender', 'Chelsea')

fixing_pos_team(df_1819, 'Fabio Henrique Tavares', 'Midfielder', 'Liverpool')
fixing_pos_team(df_1819, 'Fabricio Agosto Ramírez', 'Goalkeeper', 'Fulham')
fixing_pos_team(df_1819, 'Frederico Rodrigues de Paula Santos', 'Midfielder', 'Manchester United')
fixing_pos_team(df_1819, 'Javier Hernández Balcázar', 'Forward', 'West Ham United')
fixing_pos_team(df_1819, 'Jorge Luiz Frello Filho', 'Midfielder', 'Chelsea')
fixing_pos_team(df_1819, 'Jose Luis Mato Sanmartín', 'Forward', 'Newcastle United')
fixing_pos_team(df_1819, 'João Filipe Iria Santos Moutinho', 'Midfielder', 'Wolverhampton Wanderers')
fixing_pos_team(df_1819, 'Kepa Arrizabalaga', 'Goalkeeper', 'Chelsea')
fixing_pos_team(df_1819, 'Lucas Rodrigues Moura da Silva', 'Forward', 'Tottenham Hotspur')
fixing_pos_team(df_1819, 'Pedro Rodríguez Ledesma', 'Forward', 'Chelsea')
fixing_pos_team(df_1819, 'Robert Kenedy Nunes do Nascimento', 'Midfielder', 'Chelsea')
fixing_pos_team(df_1819, 'Rui Pedro da Rocha Fonte', 'Defender', 'Chelsea')
fixing_pos_team(df_1819, 'Rui Pedro dos Santos Patrício', 'Goalkeeper', 'Wolverhampton Wanderers')
fixing_pos_team(df_1819, 'Rúben Diogo da Silva Neves', 'Midfielder', 'Wolverhampton Wanderers')
fixing_pos_team(df_1819, 'Rúben Gonçalo Silva Nascimento Vinagre', 'Defender', 'Wolverhampton Wanderers')
fixing_pos_team(df_1819, 'Willian Borges Da Silva', 'Forward', 'Chelsea')

In [594]:
df_1819[df_1819['team'].isnull()]['name'].sort_values().unique()

array([], dtype=object)

### 2019-20

In [595]:
url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/merged_gw.csv"
df_1920 = pd.read_csv(url)

In [596]:
df_1920.head(3)

,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,Aaron_Cresswell_376,0,0,7,0,1.5,376,8,5,0,1.1,9.0,2019-08-10T11:30:00Z,90,11,0,0,0,0,1,0,23399,5.0,0.0,0.0,0,0,0,0,50,True,0,1
1,Aaron_Lennon_430,0,0,3,0,0.0,430,3,0,0,0.2,2.0,2019-08-10T14:00:00Z,6,16,0,0,0,0,1,0,8105,0.0,3.0,0.0,1,0,0,0,50,True,0,1
2,Aaron_Mooy_516,0,0,0,0,0.0,516,7,0,0,0.0,0.0,2019-08-10T14:00:00Z,0,18,0,0,0,0,1,0,16261,3.0,0.0,0.0,0,0,0,0,50,False,0,1


In [597]:
players_1920_df = players_df[players_df['season'] == '2019/20']

# Terapkan fungsi untuk setiap baris di df_1920
df_1920['name'] = df_1920['name'].apply(clean_name)
df_1920[['position', 'team']] = df_1920.apply(match_and_add_columns, players_df=players_1920_df, axis=1)

In [598]:
df_1920[df_1920['team'].isnull()]['name'].sort_values().unique()

array(['Abd-Al-Ali Morakinyo Olaposi Koiki',
       'Addji Keaninkin Marc-Israel Guehi',
       'Adrián San Miguel del Castillo',
       'Alexandre Nascimento Costa Silva', 'André Filipe Tavares Gomes',
       'Bernard Anício Caldeira Duarte',
       'Bernardo Fernandes da Silva Junior',
       'Bernardo Mota Veiga de Carvalho e Silva', 'Borja González Tomás',
       'Bruno André Cavaco Jordao', 'David Luiz Moreira Marinho',
       'Emerson Palmieri dos Santos',
       'Frederico Rodrigues de Paula Santos', 'Javier Hernández Balcázar',
       'Joelinton Cássio Apolinário de Lira', 'Jorge Luiz Frello Filho',
       'Jose Luis Mato Sanmartín', 'José Ignacio Peleteiro Romallo',
       'José Ángel Esmorís Tasende', 'João Filipe Iria Santos Moutinho',
       'João Pedro Junqueira de Jesus', 'Lucas Rodrigues Moura da Silva',
       'Mahmoud Ahmed Ibrahim Hassan', 'Pedro Rodríguez Ledesma',
       'Robert Kenedy Nunes do Nascimento',
       'Rui Pedro dos Santos Patrício', 'Rúben Diogo da Sil

In [599]:
fixing_pos_team(df_1920, 'Abd-Al-Ali Morakinyo Olaposi Koiki', 'Midfielder', 'Burnley')
fixing_pos_team(df_1920, 'Addji Keaninkin Marc-Israel Guehi', 'Defender', 'Crystal Palace')
fixing_pos_team(df_1920, 'Adrián San Miguel del Castillo', 'Goalkeeper', 'Liverpool')
fixing_pos_team(df_1920, 'Alexandre Nascimento Costa Silva', 'Forward', 'West Ham United')
fixing_pos_team(df_1920, 'André Filipe Tavares Gomes', 'Midfielder', 'Everton')

fixing_pos_team(df_1920, 'Bernard Anício Caldeira Duarte', 'Forward', 'Everton')
fixing_pos_team(df_1920, 'Bernardo Fernandes da Silva Junior', 'Defender', 'Brighton & Hove Albion')
fixing_pos_team(df_1920, 'Bernardo Mota Veiga de Carvalho e Silva', 'Midfielder', 'Manchester City')

fixing_pos_team(df_1920, 'Borja González Tomás', 'Forward', 'Aston Villa')
fixing_pos_team(df_1920, 'Bruno André Cavaco Jordao', 'Midfielder', 'Wolverhampton Wanderers')
fixing_pos_team(df_1920, 'David Luiz Moreira Marinho', 'Defender', 'Arsenal')
fixing_pos_team(df_1920, 'Ederson Santana de Moraes', 'Goalkeeper', 'Manchester City')
fixing_pos_team(df_1920, 'Emerson Palmieri dos Santos', 'Defender', 'Chelsea')

fixing_pos_team(df_1920, 'Frederico Rodrigues de Paula Santos', 'Midfielder', 'Manchester United')
fixing_pos_team(df_1920, 'Javier Hernández Balcázar', 'Forward', 'West Ham United')
fixing_pos_team(df_1920, 'Joelinton Cássio Apolinário de Lira', 'Midfielder', 'Newcastle United')
fixing_pos_team(df_1920, 'Jorge Luiz Frello Filho', 'Midfielder', 'Chelsea')

fixing_pos_team(df_1920, 'Jose Luis Mato Sanmartín', 'Forward', 'Newcastle United')
fixing_pos_team(df_1920, 'José Ignacio Peleteiro Romallo', 'Midfielder', 'Aston Villa')
fixing_pos_team(df_1920, 'José Ángel Esmorís Tasende', 'Defender', 'Manchester City')
fixing_pos_team(df_1920, 'João Filipe Iria Santos Moutinho', 'Midfielder', 'Wolverhampton Wanderers')
fixing_pos_team(df_1920, 'João Pedro Junqueira de Jesus', 'Forward', 'Watford')
fixing_pos_team(df_1920, 'Kepa Arrizabalaga', 'Goalkeeper', 'Chelsea')

fixing_pos_team(df_1920, 'Lucas Rodrigues Moura da Silva', 'Forward', 'Tottenham Hotspur')
fixing_pos_team(df_1920, 'Mahmoud Ahmed Ibrahim Hassan', 'Midfielder', 'Aston Villa')
fixing_pos_team(df_1920, 'Pedro Rodríguez Ledesma', 'Forward', 'Chelsea')
fixing_pos_team(df_1920, 'Robert Kenedy Nunes do Nascimento', 'Midfielder', 'Chelsea')

fixing_pos_team(df_1920, 'Rui Pedro da Rocha Fonte', 'Defender', 'Chelsea')
fixing_pos_team(df_1920, 'Rui Pedro dos Santos Patrício', 'Goalkeeper', 'Wolverhampton Wanderers')
fixing_pos_team(df_1920, 'Rúben Diogo da Silva Neves', 'Midfielder', 'Wolverhampton Wanderers')
fixing_pos_team(df_1920, 'Rúben Gonçalo Silva Nascimento Vinagre', 'Defender', 'Wolverhampton Wanderers')
fixing_pos_team(df_1920, 'Willian Borges Da Silva', 'Forward', 'Chelsea')

In [600]:
df_1920[df_1920['team'].isnull()]['name'].sort_values().unique()

array([], dtype=object)

### 2020-21

In [601]:
url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/gws/merged_gw.csv"
df_2021 = pd.read_csv(url)

In [602]:
players_2021_df = players_df[players_df['season'] == '2020/21']

# Terapkan fungsi untuk setiap baris di df_2021
df_2021['name'] = df_2021['name'].apply(clean_name)
df_2021[['position', 'team']] = df_2021.apply(match_and_add_columns, players_df=players_2021_df, axis=1)

In [603]:
df_2021[df_2021['team'].isnull()]['name'].sort_values().unique()

array(['Adrián San Miguel del Castillo', 'Allan Marques Loureiro',
       'André Filipe Tavares Gomes', 'Bernard Anício Caldeira Duarte',
       'Bernardo Fernandes da Silva Junior',
       'Bernardo Mota Veiga de Carvalho e Silva',
       'Bruno André Cavaco Jordao', 'David Luiz Moreira Marinho',
       'Emerson Palmieri dos Santos', 'Fabricio Agosto Ramírez',
       'Francisco Casilla Cortés', 'Frederico Rodrigues de Paula Santos',
       'Hélder Wander Sousa de Azevedo e Costa',
       'Ivan Ricardo Neves Abreu Cavaleiro',
       'Joelinton Cássio Apolinário de Lira', 'Jordan Hugill',
       'Jorge Luiz Frello Filho', 'José Ignacio Peleteiro Romallo',
       'João Filipe Iria Santos Moutinho', 'Kelland Watts',
       'Lucas Rodrigues Moura da Silva', 'Léo Bonatini',
       'Mahmoud Ahmed Ibrahim Hassan',
       'Oluwasemilogo Adesewo Ibidapo Ajayi', 'Raphael Dias Belloli',
       'Rolando Aarons', 'Rui Pedro dos Santos Patrício',
       'Rúben Diogo da Silva Neves',
       'Rúben Go

In [604]:
fixing_pos_team(df_2021, 'Adrián San Miguel del Castillo', 'Goalkeeper', 'Liverpool')
fixing_pos_team(df_2021, 'Allan Marques Loureiro', 'Midfielder', 'Everton')
fixing_pos_team(df_2021, 'André Filipe Tavares Gomes', 'Midfielder', 'Everton')
fixing_pos_team(df_2021, 'Bernard Anício Caldeira Duarte', 'Forward', 'Everton')
fixing_pos_team(df_2021, 'Bernardo Fernandes da Silva Junior', 'Defender', 'Brighton & Hove Albion')
fixing_pos_team(df_2021, 'Bernardo Mota Veiga de Carvalho e Silva', 'Midfielder', 'Manchester City')

fixing_pos_team(df_2021, 'Bruno André Cavaco Jordao', 'Midfielder', 'Wolverhampton Wanderers')
fixing_pos_team(df_2021, 'David Luiz Moreira Marinho', 'Defender', 'Arsenal')
fixing_pos_team(df_2021, 'Ederson Santana de Moraes', 'Goalkeeper', 'Manchester City')
fixing_pos_team(df_2021, 'Emerson Palmieri dos Santos', 'Defender', 'Chelsea')

fixing_pos_team(df_2021, 'Fabricio Agosto Ramírez', 'Goalkeeper', 'Fulham')
fixing_pos_team(df_2021, 'Francisco Casilla Cortés', 'Goalkeeper', 'Leeds United')
fixing_pos_team(df_2021, 'Frederico Rodrigues de Paula Santos', 'Midfielder', 'Manchester United')
fixing_pos_team(df_2021, 'Hélder Wander Sousa de Azevedo e Costa', 'Forward', 'Leeds United')

fixing_pos_team(df_2021, 'Ivan Ricardo Neves Abreu Cavaleiro', 'Forward', 'Fulham')
fixing_pos_team(df_2021, 'Joelinton Cássio Apolinário de Lira', 'Midfielder', 'Newcastle United')
fixing_pos_team(df_2021, 'Jordan Hugill', 'Forward', 'West Bromwich Albion')
fixing_pos_team(df_2021, 'Jorge Luiz Frello Filho', 'Midfielder', 'Chelsea')
fixing_pos_team(df_2021, 'José Ignacio Peleteiro Romallo', 'Midfielder', 'Aston Villa')

fixing_pos_team(df_2021, 'Jose Luis Mato Sanmartín', 'Forward', 'Newcastle United')
fixing_pos_team(df_2021, 'José Ignacio Peleteiro Romallo', 'Midfielder', 'Aston Villa')
fixing_pos_team(df_2021, 'João Filipe Iria Santos Moutinho', 'Midfielder', 'Wolverhampton Wanderers')
fixing_pos_team(df_2021, 'Kelland Watts', 'Midfielder', 'Newcastle United')
fixing_pos_team(df_2021, 'Kepa Arrizabalaga', 'Goalkeeper', 'Chelsea')
fixing_pos_team(df_2021, 'Lucas Rodrigues Moura da Silva', 'Forward', 'Tottenham Hotspur')

fixing_pos_team(df_2021, 'Léo Bonatini', 'Forward', 'Wolverhampton Wanderers')
fixing_pos_team(df_2021, 'Mahmoud Ahmed Ibrahim Hassan', 'Midfielder', 'Aston Villa')
fixing_pos_team(df_2021, 'Oluwasemilogo Adesewo Ibidapo Ajayi', 'Defender', 'West Bromwich Albion')
fixing_pos_team(df_2021, 'Raphael Dias Belloli', 'Forward', 'Leeds United')
fixing_pos_team(df_2021, 'Rolando Aarons', 'Midfielder', 'Newcastle United')
fixing_pos_team(df_2021, 'Rui Pedro dos Santos Patrício', 'Goalkeeper', 'Wolverhampton Wanderers')
fixing_pos_team(df_2021, 'Rúben Diogo da Silva Neves', 'Midfielder', 'Wolverhampton Wanderers')
fixing_pos_team(df_2021, 'Rúben Gonçalo Silva Nascimento Vinagre', 'Defender', 'Wolverhampton Wanderers')

fixing_pos_team(df_2021, 'Rúben Santos Gato Alves Dias', 'Defender', 'Manchester City')
fixing_pos_team(df_2021, 'Willian Borges Da Silva', 'Forward', 'Arsenal')

In [605]:
df_2021[df_2021['team'].isnull()]['name'].sort_values().unique()

array([], dtype=object)

### 2021-22

In [606]:
url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2021-22/gws/merged_gw.csv"
df_2122 = pd.read_csv(url)

In [607]:
df_2122.head(3)

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,Eric Bailly,DEF,Man Utd,0.0,0,0,0,0,0.0,286,6,0,0,0.0,0.0,2021-08-14T11:30:00Z,0,10,0,0,0,0,1,0,9363,1,5,0.0,0,0,0,0,50,True,0,1
1,Keinan Davis,FWD,Aston Villa,0.4,0,0,0,0,0.0,49,8,0,0,0.0,0.0,2021-08-14T14:00:00Z,0,18,0,0,0,0,1,0,169789,2,3,0.0,0,0,0,0,45,False,0,1
2,Ayotomiwa Dele-Bashiru,MID,Watford,0.0,0,0,0,0,0.0,394,8,0,0,0.0,0.0,2021-08-14T14:00:00Z,0,2,0,0,0,0,1,0,4092,2,3,0.0,0,0,0,0,45,True,0,1


In [608]:
df_2122[df_2122['team'].isnull()]['name'].sort_values().unique()

array([], dtype=object)

### 2022-23

In [609]:
url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2022-23/gws/merged_gw.csv"
df_2223 = pd.read_csv(url)

In [610]:
df_2223.head(3)

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,starts,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,Nathan Redmond,MID,Southampton,1.5,0,0,3,0,0.0,403,0.0,0.0,0.0,0.0,8,0,0,0.0,0.0,2022-08-06T14:00:00Z,1,18,0,0,0,0,1,0,5871,0,1,4,0.0,1,0,0,0,55,False,0,1
1,Junior Stanislas,MID,Bournemouth,1.1,0,0,3,0,0.0,58,0.0,0.0,0.0,0.0,2,0,0,0.0,0.0,2022-08-06T14:00:00Z,1,2,0,0,0,0,1,0,9491,0,0,2,0.0,1,0,0,0,50,True,0,1
2,Armando Broja,FWD,Chelsea,2.0,0,0,3,0,0.3,150,0.0,0.0,0.0,0.0,3,0,0,2.5,5.2,2022-08-06T16:30:00Z,15,8,0,0,0,0,1,0,124903,0,1,0,19.0,1,0,0,0,55,False,0,1


In [611]:
df_2223['team'].sort_values().unique()

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
       'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leeds',
       'Leicester', 'Liverpool', 'Man City', 'Man Utd', 'Newcastle',
       "Nott'm Forest", 'Southampton', 'Spurs', 'West Ham', 'Wolves'],
      dtype=object)

In [612]:
df_2223[df_2223['team'].isnull()]['name'].sort_values().unique()

array([], dtype=object)

### 2023-24

In [613]:
url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2023-24/gws/merged_gw.csv"
df_2324 = pd.read_csv(url)

In [614]:
df_2324.head(3)

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,starts,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,Femi Seriki,DEF,Sheffield Utd,0.5,0,0,0,0,0.0,653,0.00,0.00,0.0,0.00,7,0,0,0.0,0.0,2023-08-12T14:00:00Z,0,8,0,0,0,0,1,0,0,0,1,0,0.0,0,0,0,0,40,True,0,1
1,Jack Hinshelwood,MID,Brighton,1.5,0,0,0,0,0.0,621,0.00,0.00,0.0,0.00,4,0,0,0.0,0.0,2023-08-12T14:00:00Z,0,12,0,0,0,0,1,0,822,0,1,4,0.0,0,0,0,0,45,True,0,1
2,Jadon Sancho,MID,Man Utd,3.0,0,0,4,0,11.3,397,0.05,0.05,0.0,1.08,10,0,0,2.3,3.8,2023-08-14T19:00:00Z,22,20,0,0,0,0,1,0,83993,0,0,1,8.0,1,0,0,0,70,True,0,1


In [615]:
df_2324['team'].sort_values().unique()

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham',
       'Liverpool', 'Luton', 'Man City', 'Man Utd', 'Newcastle',
       "Nott'm Forest", 'Sheffield Utd', 'Spurs', 'West Ham', 'Wolves'],
      dtype=object)

### 2024-25

In [616]:
url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2024-25/gws/merged_gw.csv"
df_2425 = pd.read_csv(url)

In [617]:
df_2425

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,starts,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,Alex Scott,MID,Bournemouth,1.6,0,0,11,0,12.8,77,0.01,0.01,0.0,1.02,6,1,0,3.6,22.8,2024-08-17T14:00:00Z,62,16,0,0,0,0,1,0,4339,1,1,1,0.0,2,0,0,0,50,False,0,1
1,Carlos Miguel dos Santos Pereira,GK,Nott'm Forest,2.2,0,0,0,0,0.0,427,0.00,0.00,0.0,0.00,6,0,0,0.0,0.0,2024-08-17T14:00:00Z,0,3,0,0,0,0,1,0,33324,0,1,1,0.0,0,0,0,0,45,True,0,1
2,Tomiyasu Takehiro,DEF,Arsenal,0.0,0,0,0,0,0.0,22,0.00,0.00,0.0,0.00,2,0,0,0.0,0.0,2024-08-17T14:00:00Z,0,20,0,0,0,0,1,0,8462,0,0,2,0.0,0,0,0,0,50,True,0,1
3,Malcolm Ebiowei,MID,Crystal Palace,0.0,0,0,0,0,0.0,197,0.00,0.00,0.0,0.00,8,0,0,0.0,0.0,2024-08-18T13:00:00Z,0,4,0,0,0,0,1,0,716,0,1,2,0.0,0,0,0,0,45,False,0,1
4,Ben Brereton Díaz,MID,Southampton,1.0,0,0,-2,0,14.0,584,0.02,0.32,0.3,0.25,5,1,0,3.3,2.6,2024-08-17T14:00:00Z,70,15,0,0,0,0,1,0,66244,1,0,1,16.0,1,0,0,0,55,False,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,Giovani Lo Celso,MID,Spurs,0.0,0,0,0,0,0.0,493,0.00,0.00,0.0,0.00,19,0,0,0.0,0.0,2024-08-24T14:00:00Z,0,8,0,0,0,0,2,0,9366,0,0,4,0.0,0,-1213,720,1933,50,True,0,2
1239,Tawanda Chirewa,MID,Wolves,0.0,0,0,0,0,0.0,538,0.00,0.00,0.0,0.00,20,0,0,0.0,0.0,2024-08-25T13:00:00Z,0,6,0,0,0,0,2,0,8897,0,6,2,0.0,0,-295,916,1211,45,True,0,2
1240,Cameron Humphreys,MID,Ipswich,0.0,0,0,0,0,0.0,273,0.00,0.00,0.0,0.00,17,0,0,0.0,0.0,2024-08-24T14:00:00Z,0,13,0,0,0,0,2,0,5456,0,1,4,0.0,0,-300,651,951,45,False,0,2
1241,Ashley Young,DEF,Everton,0.0,0,0,0,0,0.0,238,0.00,0.00,0.0,0.00,19,0,0,0.0,0.0,2024-08-24T14:00:00Z,0,18,0,0,0,0,2,0,13810,0,0,4,0.0,0,-6644,181,6825,45,False,0,2


## Combine & Formatting Features

In [618]:
new_features_list = ['xP', 'expected_assists', 'expected_goal_involvements', 'expected_goals', 'expected_goals_conceded']
df_list = [df_1617, df_1718, df_1819, df_1920]

for df  in df_list:
    for feature in new_features_list:
        df[feature] = 0

In [619]:
df_list = [df_1617, df_1718, df_1819, df_1920, df_2021, df_2122, df_2223, df_2324, df_2425]
season_list = ['16/17', '17/18', '18/19', '19/20', '20/21', '21/22', '22/23', '23/24', '24/25']

for df, season in zip(df_list, season_list):
    df['season'] = f'20{season}'

In [620]:
df_list = [df_2021, df_2122]
new_features_list = ['expected_assists', 'expected_goal_involvements', 'expected_goals', 'expected_goals_conceded']

for df in df_list:
    for feature in new_features_list:
        df[feature] = 0

In [621]:
cols = ['season', 'name', 'position', 'team', 'assists', 'bonus', 'bps',
        'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded',
        'goals_scored', 'ict_index', 'influence', 'xP', 'expected_assists', 
        'expected_goal_involvements', 'expected_goals', 'expected_goals_conceded', 
        'kickoff_time', 'minutes', 'opponent_team', 'own_goals', 'penalties_missed', 
        'penalties_saved', 'red_cards', 'saves', 'selected', 'team_a_score',
        'team_h_score', 'threat', 'total_points', 'transfers_balance', 'transfers_in', 
        'transfers_out', 'value', 'was_home', 'yellow_cards', 'GW']

df_1617 = df_1617[cols]
df_1718 = df_1718[cols]
df_1819 = df_1819[cols]
df_1920 = df_1920[cols]
df_2021 = df_2021[cols]
df_2122 = df_2122[cols]
df_2223 = df_2223[cols]
df_2324 = df_2324[cols]
df_2425 = df_2425[cols]

In [622]:
df = pd.concat([df_1617, df_1718, df_1819, df_1920, df_2021, df_2122, df_2223, df_2324, df_2425]).reset_index(drop=True)
df.head()

,season,name,position,team,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,xP,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,2016/17,Aaron Cresswell,Defender,West Ham United,0,0,0,0,0.0,454,10,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-08-15T19:00:00Z,0,4,0,0,0,0,0,14023,1.0,2.0,0.0,0,0,0,0,55,False,0,1
1,2016/17,Aaron Lennon,Midfielder,Everton,0,0,6,0,0.3,142,3,0,0,0.9,8.2,0.0,0.0,0.0,0.0,0.0,2016-08-13T14:00:00Z,15,17,0,0,0,0,0,13918,1.0,1.0,0.0,1,0,0,0,60,True,0,1
2,2016/17,Aaron Ramsey,Midfielder,Arsenal,0,0,5,0,4.9,16,8,3,0,3.0,2.2,0.0,0.0,0.0,0.0,0.0,2016-08-14T15:00:00Z,60,9,0,0,0,0,0,163170,4.0,3.0,23.0,2,0,0,0,80,True,0,1
3,2016/17,Abdoulaye Doucouré,Midfielder,Watford,0,0,0,0,0.0,482,7,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-08-13T14:00:00Z,0,13,0,0,0,0,0,1051,1.0,1.0,0.0,0,0,0,0,50,False,0,1
4,2016/17,Abdul Rahman Baba,Defender,Chelsea,0,0,0,0,0.0,80,10,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-08-15T19:00:00Z,0,20,0,0,0,0,0,1243,1.0,2.0,0.0,0,0,0,0,55,True,0,1


In [623]:
df.loc[df['name'].str.contains('Frello'), 'name'] = 'Jorginho'

In [624]:
team_map = {
    'Arsenal':'Arsenal',
    'Aston Villa':'Aston Villa',
    'Brentford':'Brentford',
    'Bournemouth':'Bournemouth',
    'Brighton':'Brighton & Hove Albion',
    'Burnley':'Burnley',
    'Chelsea':'Chelsea',
    'Cardiff':'Cardiff City',
    'Crystal Palace':'Crystal Palace',
    'Everton':'Everton',
    'Hull':'Hull City',
    'Huddersfield':'Huddersfield Town',
    'Leeds':'Leeds United',
    'Leicester':'Leicester City',
    'Ipswich': 'Ipswich Town',
    'Liverpool':'Liverpool',
    'Luton':'Luton Town',
    'Man City':'Manchester City',
    'Man Utd':'Manchester United',
    'Middlesbrough':'Middlesbrough',
    'Newcastle':'Newcastle United',
    'Norwich':'Norwich City',
    "Nott'm Forest":'Nottingham Forest',
    'Southampton':'Southampton',
    'Swansea':'Swansea City',
    'Stoke':'Stoke City',
    'Sunderland':'Sunderland',
    'Sheffield Utd':'Sheffield United',
    'Spurs':'Tottenham Hotspur',
    'Watford':'Watford',
    'West Ham':'West Ham United',
    'West Brom':'West Bromwich United',
    'Wolves':'Wolverhampton Wanderers'
}

pos_map = {
    'Defender':'DEF',
    'Midfielder':'MID',
    'Forward':'FWD',
    'Goalkeeper':'GK',
    'GKP':'GK'
}


df['position'] = df['position'].replace(pos_map)
df['team'] = df['team'].replace(team_map)

In [625]:
url_master_team = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/master_team_list.csv"
team_id = pd.read_csv(url_master_team)

In [626]:
id_team_1617 = team_id[team_id['season'] == '2016-17']
id_team_1718 = team_id[team_id['season'] == '2017-18']
id_team_1819 = team_id[team_id['season'] == '2018-19']
id_team_1920 = team_id[team_id['season'] == '2019-20']
id_team_2021 = team_id[team_id['season'] == '2020-21']
id_team_2122 = team_id[team_id['season'] == '2021-22']
id_team_2223 = team_id[team_id['season'] == '2022-23']
id_team_2324 = team_id[team_id['season'] == '2023-24']

In [627]:
team_id_1617_map = dict(zip(id_team_1617['team'], id_team_1617['team_name']))
team_id_1718_map = dict(zip(id_team_1718['team'], id_team_1718['team_name']))
team_id_1819_map = dict(zip(id_team_1819['team'], id_team_1819['team_name']))
team_id_1920_map = dict(zip(id_team_1920['team'], id_team_1920['team_name']))
team_id_2021_map = dict(zip(id_team_2021['team'], id_team_2021['team_name']))
team_id_2122_map = dict(zip(id_team_2122['team'], id_team_2122['team_name']))
team_id_2223_map = dict(zip(id_team_2223['team'], id_team_2223['team_name']))
team_id_2324_map = dict(zip(id_team_2324['team'], id_team_2324['team_name']))

df.loc[df['season'] == '2016/17', 'opponent_team'] = df.loc[df['season'] == '2016/17', 'opponent_team'].replace(team_id_1617_map)
df.loc[df['season'] == '2017/18', 'opponent_team'] = df.loc[df['season'] == '2017/18', 'opponent_team'].replace(team_id_1718_map)
df.loc[df['season'] == '2018/19', 'opponent_team'] = df.loc[df['season'] == '2018/19', 'opponent_team'].replace(team_id_1819_map)
df.loc[df['season'] == '2019/20', 'opponent_team'] = df.loc[df['season'] == '2019/20', 'opponent_team'].replace(team_id_1920_map)
df.loc[df['season'] == '2020/21', 'opponent_team'] = df.loc[df['season'] == '2020/21', 'opponent_team'].replace(team_id_2021_map)
df.loc[df['season'] == '2021/22', 'opponent_team'] = df.loc[df['season'] == '2021/22', 'opponent_team'].replace(team_id_2122_map)
df.loc[df['season'] == '2022/23', 'opponent_team'] = df.loc[df['season'] == '2022/23', 'opponent_team'].replace(team_id_2223_map)
df.loc[df['season'] == '2023/24', 'opponent_team'] = df.loc[df['season'] == '2023/24', 'opponent_team'].replace(team_id_2324_map)

C:\Users\marse\AppData\Local\Temp\ipykernel_27432\3256549819.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Chelsea' 'Spurs' 'Liverpool' ... 'Chelsea' 'Sunderland' 'Arsenal']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['season'] == '2016/17', 'opponent_team'] = df.loc[df['season'] == '2016/17', 'opponent_team'].replace(team_id_1617_map)


In [628]:
fixtures_2425 = pd.read_csv('../../data/fpl/combined-fpl-price.csv')

opponent_team_map = {
    (row['team'], row['GW']): row['opponent_team'] 
    for _, row in fixtures_2425.iterrows()
}

# Fungsi untuk mendapatkan nilai opponent_team berdasarkan team dan GW
def update_opponent_team(row):
    return opponent_team_map.get((row['team'], row['GW']), row['opponent_team'])

df['opponent_team'] = df.apply(update_opponent_team, axis=1)
df['opponent_team'] = df['opponent_team'].replace(team_map)

In [629]:
df

,season,name,position,team,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,xP,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,2016/17,Aaron Cresswell,DEF,West Ham United,0,0,0,0,0.0,454,10,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-08-15T19:00:00Z,0,Aston Villa,0,0,0,0,0,14023,1.0,2.0,0.0,0,0,0,0,55,False,0,1
1,2016/17,Aaron Lennon,MID,Everton,0,0,6,0,0.3,142,3,0,0,0.9,8.2,0.0,0.0,0.0,0.0,0.0,2016-08-13T14:00:00Z,15,Brighton & Hove Albion,0,0,0,0,0,13918,1.0,1.0,0.0,1,0,0,0,60,True,0,1
2,2016/17,Aaron Ramsey,MID,Arsenal,0,0,5,0,4.9,16,8,3,0,3.0,2.2,0.0,0.0,0.0,0.0,0.0,2016-08-14T15:00:00Z,60,Wolverhampton Wanderers,0,0,0,0,0,163170,4.0,3.0,23.0,2,0,0,0,80,True,0,1
3,2016/17,Abdoulaye Doucouré,MID,Watford,0,0,0,0,0.0,482,7,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-08-13T14:00:00Z,0,Southampton,0,0,0,0,0,1051,1.0,1.0,0.0,0,0,0,0,50,False,0,1
4,2016/17,Abdul Rahman Baba,DEF,Chelsea,0,0,0,0,0.0,80,10,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-08-15T19:00:00Z,0,Manchester City,0,0,0,0,0,1243,1.0,2.0,0.0,0,0,0,0,55,True,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197776,2024/25,Giovani Lo Celso,MID,Tottenham Hotspur,0,0,0,0,0.0,493,19,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-08-24T14:00:00Z,0,Everton,0,0,0,0,0,9366,0.0,4.0,0.0,0,-1213,720,1933,50,True,0,2
197777,2024/25,Tawanda Chirewa,MID,Wolverhampton Wanderers,0,0,0,0,0.0,538,20,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-08-25T13:00:00Z,0,Chelsea,0,0,0,0,0,8897,6.0,2.0,0.0,0,-295,916,1211,45,True,0,2
197778,2024/25,Cameron Humphreys,MID,Ipswich Town,0,0,0,0,0.0,273,17,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-08-24T14:00:00Z,0,Manchester City,0,0,0,0,0,5456,1.0,4.0,0.0,0,-300,651,951,45,False,0,2
197779,2024/25,Ashley Young,DEF,Everton,0,0,0,0,0.0,238,19,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-08-24T14:00:00Z,0,Tottenham Hotspur,0,0,0,0,0,13810,0.0,4.0,0.0,0,-6644,181,6825,45,False,0,2


In [630]:
df[(df['saves'] > 0) & (df['position'].isin(['MID', 'FWD', 'DEF']))].head(2)

,season,name,position,team,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,xP,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
80370,2019/20,José Reina,DEF,Watford,0,0,11,0,0.0,603,222,1,0,1.4,13.8,0.0,0.0,0.0,0.0,0.0,2020-01-18T15:00:00Z,90,Brighton & Hove Albion,0,0,0,0,1,10237,1.0,1.0,0.0,2,9786,10752,966,45,False,0,23
81125,2019/20,José Reina,DEF,Watford,0,0,12,0,0.0,603,231,1,0,1.5,14.6,0.0,0.0,0.0,0.0,0.0,2020-01-21T19:30:00Z,90,Watford,0,0,0,0,1,18885,1.0,2.0,0.0,2,7499,9066,1567,45,True,0,24


In [631]:
df.loc[((df['name'] == 'José Reina') & (df['season'] == '2019/20')), ['position', 'team']] = ['GK', 'Aston Villa']

In [632]:
df['team'].sort_values().unique()

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford',
       'Brighton & Hove Albion', 'Burnley', 'Cardiff City', 'Chelsea',
       'Crystal Palace', 'Everton', 'Fulham', 'Huddersfield Town',
       'Hull City', 'Ipswich Town', 'Leeds United', 'Leicester City',
       'Liverpool', 'Luton Town', 'Manchester City', 'Manchester United',
       'Middlesbrough', 'Newcastle United', 'Norwich City',
       'Nottingham Forest', 'Sheffield United', 'Southampton',
       'Stoke City', 'Sunderland', 'Swansea City', 'Tottenham Hotspur',
       'Watford', 'West Bromwich Albion', 'West Ham United',
       'Wolverhampton Wanderers'], dtype=object)

## Correcting & Formatting 'name' players

In [633]:
players_info_df = pd.read_csv('../../data/processed/players_information.csv')
players_info_df = players_info_df.replace(pos_map)
players_info_df = players_info_df.dropna()

In [634]:
from rapidfuzz import fuzz, process
import pandas as pd

# Fungsi untuk membalik nama pemain Korea Selatan
def reverse_korean_name(name):
    parts = name.split(' ')
    if len(parts) == 2:
        return f"{parts[1]} {parts[0]}"
    return name

# Terapkan pada player_info untuk pemain dengan kewarganegaraan 'South Korea'
def apply_reverse_to_korean_players(player_info):
    korean_players = player_info['nationality'] == 'South Korea'
    player_info.loc[korean_players, 'name'] = player_info.loc[korean_players, 'name'].apply(reverse_korean_name)
    return player_info

# Fungsi untuk mencocokkan nama dengan logika berdasarkan 'team' atau 'name'
def find_best_match(row, players_name):
    # Filter berdasarkan season
    subset = players_name[players_name['season'] == row['season']]
    
    # Pencarian berdasarkan 'team' terlebih dahulu
    team_subset = subset[subset['team'] == row['team']]
    
    # Aturan pencocokan posisi untuk tim yang sama
    if row['position'] == 'FWD':
        position_subset = team_subset[team_subset['position'].isin(['FWD', 'MID'])]
    elif row['position'] == 'MID':
        position_subset = team_subset[team_subset['position'].isin(['MID', 'FWD'])]
    else:
        position_subset = team_subset[team_subset['position'] == row['position']]

    # Cek apakah ada kecocokan pada 'team' yang sama
    if not position_subset.empty:
        best_match = process.extractOne(row['name'], position_subset['name'], scorer=fuzz.token_sort_ratio)
        if best_match and best_match[1] > 60:
            return best_match[0]

    # Jika tidak ada kecocokan pada 'team', cari kecocokan hanya berdasarkan 'name' dan 'position'
    if row['position'] == 'FWD':
        position_subset = players_name[players_name['position'].isin(['FWD', 'MID'])]
    elif row['position'] == 'MID':
        position_subset = players_name[players_name['position'].isin(['MID', 'FWD'])]
    else:
        position_subset = players_name[players_name['position'] == row['position']]

    # Cari kecocokan hanya berdasarkan 'name' tanpa memperhatikan 'team' dan 'season'
    best_match = process.extractOne(row['name'], position_subset['name'], scorer=fuzz.token_sort_ratio)
    if best_match and best_match[1] > 30:
        return best_match[0]

    # Jika tidak ada kecocokan, kembalikan None untuk menghapus nama
    return None

# Terapkan fungsi pembalikan nama Korea pada dataset players_info_df
players_info_df = apply_reverse_to_korean_players(players_info_df)

# Terapkan fungsi pencocokan ke df untuk menemukan nama terbaik berdasarkan aturan pencocokan
df['name'] = df.apply(find_best_match, axis=1, players_name=players_info_df)

In [636]:
df[(df['position'].str.contains('GK')) & (df['team'].str.contains('Manchester City'))]

,season,name,position,team,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,xP,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
255,2016/17,Joe Hart,GK,Manchester City,0,0,0,0,0.0,218,5,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,2016-08-13T16:30:00Z,0,Chelsea,0,0,0,0,0,200797,1.0,2.0,0.0,0,0,0,0,55,True,0,1
514,2016/17,Willy Caballero,GK,Manchester City,0,0,12,0,0.0,219,5,1,0,1.7,17.4,0.0,0.0,0.0,0.0,0.00,2016-08-13T16:30:00Z,90,Chelsea,0,0,0,0,2,5495,1.0,2.0,0.0,2,0,0,0,50,True,0,1
637,2016/17,Claudio Bravo,GK,Manchester City,0,0,25,1,0.0,541,100,0,0,0.9,9.2,0.0,0.0,0.0,0.0,0.00,2016-10-29T14:00:00Z,90,West Bromwich United,0,0,0,0,1,101138,4.0,0.0,0.0,6,-4834,1793,6627,55,False,0,10
813,2016/17,Joe Hart,GK,Manchester City,0,0,0,0,0.0,218,100,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,2016-10-29T14:00:00Z,0,West Bromwich United,0,0,0,0,0,50914,4.0,0.0,0.0,0,-1069,10,1079,53,False,0,10
1110,2016/17,Willy Caballero,GK,Manchester City,0,0,0,0,0.0,219,100,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,2016-10-29T14:00:00Z,0,West Bromwich United,0,0,0,0,0,16224,4.0,0.0,0.0,0,-553,45,598,49,False,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196917,2024/25,Ederson Moraes,GK,Manchester City,0,1,30,1,0.0,347,9,0,0,2.0,19.6,5.0,0.0,0.0,0.0,1.01,2024-08-18T15:30:00Z,90,Chelsea,0,0,0,0,3,557560,2.0,0.0,0.0,8,0,0,0,55,False,0,1
197124,2024/25,Stefan Ortega,GK,Manchester City,0,0,0,0,0.0,358,9,0,0,0.0,0.0,5.0,0.0,0.0,0.0,0.00,2024-08-18T15:30:00Z,0,Chelsea,0,0,0,0,0,77703,2.0,0.0,0.0,0,0,0,0,55,False,0,1
197278,2024/25,Scott Carson,GK,Manchester City,0,0,0,0,0.0,344,17,0,0,0.0,0.0,1.5,0.0,0.0,0.0,0.00,2024-08-24T14:00:00Z,0,Ipswich Town,0,0,0,0,0,17745,1.0,4.0,0.0,0,-40,1270,1310,40,True,0,2
197540,2024/25,Ederson Moraes,GK,Manchester City,0,0,3,0,0.0,347,17,1,0,0.4,3.8,6.5,0.0,0.0,0.0,0.29,2024-08-24T14:00:00Z,90,Ipswich Town,0,0,0,0,0,817354,1.0,4.0,0.0,2,74954,105221,30267,55,True,0,2


In [642]:
df.to_csv('../../data/processed/final-dataset.csv', index=False)